In [1]:
import cred_pg as c
import pandas as pd
from sqlalchemy import create_engine, text
from graphql.language import parse

engine_postgres = create_engine(
    f'postgresql+psycopg://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}',
    connect_args={'options': '-c search_path=$user,ugeobln,umisc,umobility,usozmed,public', 'keepalives_idle': 120},
    pool_size=1,
    max_overflow=0,
    execution_options={'isolation_level': 'AUTOCOMMIT'}
)


def perform_query(query):
    with engine_postgres.connect() as con:
        df = pd.read_sql_query(query, con)
        return df


def perform_query_json(query):
    with engine_postgres.connect() as con:
        from sqlalchemy.sql import text
        result = con.execute(text(query))
        json_result = result.fetchone()[0]
        return json_result


def get_foreign_keys():
    query = """
    SELECT con.conname AS constraint_name,
        nsp.nspname AS schema_name,
        conrel.relname AS table_name,
        att2.attname AS column_name,
        confrel.relname AS referenced_table,
        att.attname AS referenced_column
    FROM pg_constraint con
    INNER JOIN pg_class conrel ON con.conrelid = conrel.oid
    INNER JOIN pg_namespace nsp ON conrel.relnamespace = nsp.oid
    INNER JOIN pg_attribute att2 ON att2.attnum = ANY (con.conkey) AND att2.attrelid = conrel.oid
    INNER JOIN pg_class confrel ON con.confrelid = confrel.oid
    INNER JOIN pg_attribute att ON att.attnum = ANY (con.confkey) AND att.attrelid = confrel.oid
    WHERE con.contype = 'f'  -- Foreign key constraint
        AND nsp.nspname = 'ugm';  -- Replace with your schema name
    """
    df = perform_query(query)

    # Dataframe zu Dict, deduplizieren und einfache lookups
    dict_fk = {}
    for _, row in df.iterrows():
        source_table = row['table_name']
        target_table = row['referenced_table']
        target_column = row['referenced_column']

        if source_table not in dict_fk:
            dict_fk[source_table] = []

        dict_fk[source_table].append((target_table, target_column))

    return dict_fk


def get_selections_join(field, root_table=None, fk_dict=None):
    fields = []
    join_clauses = []

    # Teste ob Field selections hat
    if not hasattr(field, 'selection_set') or not field.selection_set or not field.selection_set.selections:
        return fields, join_clauses

    # Wenn keine root_table gesetzt, ist derzeitiges Element oberstes Element
    if not root_table:
        # Iteriere ueber sections
        for sub_field in field.selection_set.selections:
            # Wenn sub_field selbst keine sections hat, ist es einfache selection
            if not hasattr(sub_field,
                           'selection_set') or not sub_field.selection_set or not sub_field.selection_set.selections:
                fields.append(f"{field.name.value}.{sub_field.name.value}")
            # Wenn sub_field selbst sections hat, ist unter selection vorhanden
            else:
                # REKURSION!
                nested_fields, nested_joins = get_selections_join(sub_field, root_table=field.name.value,
                                                                  fk_dict=fk_dict)
                fields.extend(nested_fields)
                join_clauses.extend(nested_joins)

    # root_table ist gesetzt, hier Rekursion durchführen!
    else:
        join_table = field.name.value
        join_fields = [f"{join_table}.{sub_field.name.value}" for sub_field in field.selection_set.selections]

        # versuchen korrekte JOIN zu finden
        fk_found = False
        for source_table, fks in fk_dict.items():
            for target_table, target_column in fks:
                if source_table == root_table and target_table == join_table:
                    join_clauses.append(
                        f"LEFT JOIN ugm.{join_table} ON ugm.{root_table}.{target_column} = ugm.{join_table}.{target_column}")
                    fk_found = True
                    break
                elif source_table == join_table and target_table == root_table:
                    join_clauses.append(
                        f"LEFT JOIN ugm.{join_table} ON ugm.{root_table}.{target_column} = ugm.{join_table}.{target_column}")
                    fk_found = True
                    break

            if fk_found:
                break

        fields.extend(join_fields)

    return fields, join_clauses


def get_wheres(arguments, table_name):
    where_conditions = []
    for arg in arguments:
        value = arg.value.value
        where_conditions.append(f"ugm.{table_name}.{arg.name.value}='{value}'")
    return where_conditions


def graphql_eval(query):
    ast = parse(query)
    fk_dict = get_foreign_keys()

    table_name = ""
    fields = []
    where_conditions = []
    join_clauses = []

    for definition in ast.definitions:
        if not hasattr(definition, 'selection_set') or not definition.selection_set.selections:
            continue

        for selection in definition.selection_set.selections:
            table_name = selection.name.value

            selection_fields, selection_joins = get_selections_join(selection, fk_dict=fk_dict)
            fields.extend(selection_fields)
            join_clauses.extend(selection_joins)

            if selection.arguments:
                where_conditions.extend(get_wheres(selection.arguments, table_name))

            # Hier genestete selections und argumente
            for sub_selection in selection.selection_set.selections:
                if sub_selection.arguments:
                    where_conditions.extend(get_wheres(sub_selection.arguments, sub_selection.name.value))

    # SQL query bauen
    fields_str = ", ".join(fields)
    where_str = " AND ".join(where_conditions)
    join_str = " ".join(join_clauses)
    query = f"""SELECT jsonb_pretty(
                jsonb_agg(to_jsonb(t))
                )
                FROM (
                    SELECT {fields_str}
                    FROM ugm.{table_name} {join_str}"""
    if where_str:
        query += f" WHERE {where_str}"
    query += ") t"

    result = perform_query_json(query)
    print(result)
    # return result
    # return query

In [2]:
# Query 1
graphql_eval("""
query {
  kunde(kid: "10001") {
    name
    titel
  }
}
""")

[
    {
        "name": "Eck",
        "titel": "Dr."
    }
]


In [3]:
# Query 2
graphql_eval("""
query {
  kunde(kid: "10001") {
    name
    bestellung {
      bestelldatum
      gesamtpreis
    }
  }
}
""")

[
    {
        "name": "Eck",
        "gesamtpreis": 725.25,
        "bestelldatum": "2018-06-27"
    },
    {
        "name": "Eck",
        "gesamtpreis": 145.99,
        "bestelldatum": "2019-01-31"
    },
    {
        "name": "Eck",
        "gesamtpreis": 28.00,
        "bestelldatum": "2019-04-17"
    },
    {
        "name": "Eck",
        "gesamtpreis": 871.66,
        "bestelldatum": "2019-06-28"
    },
    {
        "name": "Eck",
        "gesamtpreis": 343.94,
        "bestelldatum": "2019-10-28"
    },
    {
        "name": "Eck",
        "gesamtpreis": 36.00,
        "bestelldatum": "2019-11-01"
    }
]


In [4]:
# Query 3
graphql_eval("""
query {
  kunde(kid: "10001") {
    name
    bestellung (bestelldatum: "2019-01-31") {
      bestelldatum
      gesamtpreis
    }
  }
}
""")

[
    {
        "name": "Eck",
        "gesamtpreis": 145.99,
        "bestelldatum": "2019-01-31"
    }
]


In [5]:
# Query 4
graphql_eval("""
query {
  kunde(kid: "10001") {
    name
    bestellung {
      produkt {
          bez
          laenge
      }
      bestelldatum
      gesamtpreis
    }
  }
}
""")

ProgrammingError: (psycopg.errors.UndefinedColumn) column bestellung.produkt does not exist
LINE 5:                     SELECT kunde.name, bestellung.produkt, b...
                                               ^
[SQL: SELECT jsonb_pretty(
                jsonb_agg(to_jsonb(t))
                )
                FROM (
                    SELECT kunde.name, bestellung.produkt, bestellung.bestelldatum, bestellung.gesamtpreis
                    FROM ugm.kunde LEFT JOIN ugm.bestellung ON ugm.kunde.kid = ugm.bestellung.kid WHERE ugm.kunde.kid='10001') t]
(Background on this error at: https://sqlalche.me/e/20/f405)